---

**Unleash Your Creativity: Image-to-Image Generation with Stable Diffusion**

Hello and welcome! Whether you're an artist, engineer, or simply someone curious about the blend of art and technology, this is the place for you.

Imagine taking a picture and a few words of inspiration, and watching as they transform into a new visual representation. With Stable Diffusion, this imaginative exercise becomes a tangible experience. Want to see a landscape shift from day to night? Or perhaps turn a doodle into a detailed artwork? Dive in to explore these possibilities and more.

Behind the scenes, we're harnessing the power of Intel® Data Center GPU Max Series GPUs. These GPUs are well-suited to handle the demands of tasks like Stable Diffusion, ensuring a smooth experience for you.

This guide is designed to be straightforward and user-friendly. We'll introduce you to examples, help you understand the results, and offer ways to tweak the process. No deep technical knowledge required, just a spark of creativity and a dash of curiosity.

Ready to get started? Let's begin by setting up a few things and diving into the experience!


In [70]:
# Required packages, install if not installed (assume PyTorch* and Intel® Extension for PyTorch* is already present
#!pip install diffusers accelerate transformers validators ipywidgets tensorboardX
import os
import warnings

# Suppress warnings for a cleaner output.
warnings.filterwarnings("ignore")

import random
import requests
import torch
import intel_extension_for_pytorch as ipex  # adds xpu namespace to PyTorch, enabling you to use Intel GPUs

from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionImg2ImgPipeline
from diffusers import DPMSolverMultistepScheduler
import torch.nn as nn
import time
from typing import List, Dict, Tuple
import validators
import numpy as np


---

**A Peek Under the Hood**

For those curious about how all of this works, here"s a deeper dive into the code. Don"t worry if you"re not tech-savvy; you don"t need to understand this to use the notebook. But for those interested, let"s explore:

- **Class Definition**: We"ve defined a class `Img2ImgModel` that does the heavy lifting. It sets up and optimizes the Stable Diffusion model for image transformations based on text prompts.
  
- **Initialization**: When creating an instance of this class, we can specify various parameters like the model"s path, device to run on (defaulting to Intel"s "xpu" device), and data type. We"ve also included options to optimize the model for performance and warm it up for faster initial runs.
  
- **Pipeline Loading**: The `_load_pipeline` method sets up the necessary processes (or pipeline) for our image transformations using the specified pre-trained model.
  
- **Optimization**: For best performance, the model is optimized using Intel-specific techniques in the `_optimize_pipeline` and `optimize_pipeline` methods using Intel Extension For PyTorch.
  
- **Image Handling**: Methods like `get_image_from_url` allow for fetching and processing images directly from web URLs.
  
- **Model Warmup**: Before diving into the main tasks, we "warm up" the model using the `warmup_model` method. This ensures that subsequent runs are faster.
  
- **Image Generation**: The heart of this class, `generate_images`, takes in a text prompt and an image URL. It then generates new images based on the text"s instructions, applying various parameters like strength of transformation and guidance scale.

Feel free to explore the code below, and if you"re inclined, you can see how we"ve implemented and optimized it for Intel GPUs.

---



In [73]:
class Img2ImgModel:
    """
    This class creates a model for transforming images based on given prompts.
    """

    def __init__(
        self,
        model_id_or_path: str,
        device: str = "xpu",
        torch_dtype: torch.dtype = torch.bfloat16,
        optimize: bool = True,
        warmup: bool = True,
        scheduler: bool = True,
    ) -> None:
        """
        Initialize the model with the specified parameters.

        Args:
            model_id_or_path (str): The ID or path of the pre-trained model.
            device (str, optional): The device to run the model on. Defaults to "xpu".
            torch_dtype (torch.dtype, optional): The data type to use for the model. Defaults to torch.float16.
            optimize (bool, optional): Whether to optimize the model. Defaults to True.
        """
        self.device = device
        self.data_type = torch_dtype
        self.scheduler = scheduler
        self.generator = torch.Generator()  # .manual_seed(99)
        self.pipeline = self._load_pipeline(model_id_or_path, torch_dtype)
        if optimize:
            start_time = time.time()
            print("Optimizing the model...")
            self.optimize_pipeline()
            print(
                "Optimization completed in {:.2f} seconds.".format(
                    time.time() - start_time
                )
            )
        if warmup:
            self.warmup_model()

    def _load_pipeline(
        self, model_id_or_path: str, torch_dtype: torch.dtype
    ) -> StableDiffusionImg2ImgPipeline:
        """
        Load the pipeline for the model.

        Args:
            model_id_or_path (str): The ID or path of the pre-trained model.
            torch_dtype (torch.dtype): The data type to use for the model.

        Returns:
            StableDiffusionImg2ImgPipeline: The loaded pipeline.
        """
        print("Loading the model...")
        pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
            model_id_or_path, torch_dtype=torch_dtype
        )
        pipeline = pipeline.to(self.device)
        if self.scheduler:
            pipeline.scheduler = DPMSolverMultistepScheduler.from_config(
                pipeline.scheduler.config
            )
        print("Model loaded.")
        return pipeline

    def _optimize_pipeline(
        self, pipeline: StableDiffusionImg2ImgPipeline
    ) -> StableDiffusionImg2ImgPipeline:
        """
        Optimize the pipeline of the model.

        Args:
            pipeline (StableDiffusionImg2ImgPipeline): The pipeline to optimize.

        Returns:
            StableDiffusionImg2ImgPipeline: The optimized pipeline.
        """
        for attr in dir(pipeline):
            if isinstance(getattr(pipeline, attr), nn.Module):
                setattr(
                    pipeline,
                    attr,
                    ipex.optimize(
                        getattr(pipeline, attr).eval(),
                        dtype=pipeline.text_encoder.dtype,
                        inplace=True,
                    ),
                )
        return pipeline

    def optimize_pipeline(self) -> None:
        """
        Optimize the pipeline of the model.
        """
        self.pipeline = self._optimize_pipeline(self.pipeline)

    def get_image_from_url(self, url: str, path: str) -> Image.Image:
        """
        Get an image from a URL or from a local path if it exists.

        Args:
            url (str): The URL of the image.
            path (str): The local path of the image.

        Returns:
            Image.Image: The loaded image.
        """
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception(
                f"Failed to download image. Status code: {response.status_code}"
            )
        if not response.headers["content-type"].startswith("image"):
            raise Exception(
                f"URL does not point to an image. Content type: {response.headers['content-type']}"
            )
        img = Image.open(BytesIO(response.content)).convert("RGB")
        img.save(path)
        img = img.resize((768, 512))
        return img

    def warmup_model(self):
        """
        Warms up the model by generating a sample image.
        """
        print("Setting up model...")
        start_time = time.time()
        image_url = "https://user-images.githubusercontent.com/786476/256401499-f010e3f8-6f8d-4e9f-9d1f-178d3571e7b9.png"
        try:
            self.generate_images(
                image_url=image_url,
                prompt="A beautiful day",
                num_images=1,
                save_path="/tmp",
            )
        except Exception:
            print("model warmup delayed...")
        print(
            "Model is set up and ready! Warm-up completed in {:.2f} seconds.".format(
                time.time() - start_time
            )
        )

    def get_inputs(self, prompt, batch_size=1):
        self.generator = [torch.Generator() for i in range(batch_size)]
        prompts = batch_size * [prompt]
        return {"prompt": prompts, "generator": self.generator}

    def generate_images(
        self,
        prompt: str,
        image_url: str,
        num_images: int = 5,
        num_inference_steps: int = 30,
        strength: float = 0.75,
        guidance_scale: float = 7.5,
        save_path: str = "output",
        batch_size: int = 1,
    ):
        """
        Generate images based on the provided prompt and variations.

        Args:
            prompt (str): The base prompt for the generation.
            image_url (str): The URL of the seed image.
            variations (List[str]): The list of variations to apply to the prompt.
            num_images (int, optional): The number of images to generate. Defaults to 5.
            num_inference_steps (int, optional): Number of noise removal steps.
            strength (float, optional): The strength of the transformation. Defaults to 0.75.
            guidance_scale (float, optional): The scale of the guidance. Defaults to 7.5.
            save_path (str, optional): The path to save the generated images. Defaults to "output".

        """
        input_image_path = "input.png"
        init_image = self.get_image_from_url(image_url, input_image_path)
        init_images = [init_image for _ in range(batch_size)]
        for i in range(0, num_images, batch_size):
            with torch.xpu.amp.autocast(
                enabled=True if self.data_type != torch.float32 else False,
                dtype=self.data_type,
            ):
                if batch_size > 1:
                    inputs = self.get_inputs(batch_size=batch_size, prompt=prompt)
                    images = self.pipeline(
                        **inputs,
                        image=init_images,
                        strength=strength,
                        guidance_scale=guidance_scale,
                        num_inference_steps=num_inference_steps,
                    ).images
                else:
                    images = self.pipeline(
                        prompt=prompt,
                        image=init_images,
                        strength=strength,
                        guidance_scale=guidance_scale,
                        num_inference_steps=num_inference_steps,
                    ).images

                for j in range(len(images)):
                    output_image_path = os.path.join(
                        save_path,
                        f"{'_'.join(prompt.split()[:3])}_{i+j}.png",
                    )
                    images[j].save(output_image_path)



---

**Setting Up the User Interface**

In the next section, we"ll craft an interactive user interface right here in the notebook. This will allow you to easily select a model, provide an image URL, type in a text prompt, and define other parameters without diving into the code itself.

- **Model Selection**: Choose from available pre-trained models.
- **Prompt Input**: Type in a text prompt to guide the image transformation.
- **Number of Images**: Decide how many images you"d like to generate.
- **Image URL**: Provide a link to an online image or use the default provided.
- **Enhancement**: Opt for auto-enhancements to the prompt for added creativity.

Once you"ve provided your preferences, a button click will initiate the magic!

Let"s set this up:



In [9]:
import os
import random
import time

import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mp_img
import validators

from IPython.display import clear_output
from IPython.display import display
from IPython.display import HTML
from IPython.display import Image as IPImage
from ipywidgets import VBox, HBox

def image_to_image():
    out = widgets.Output()
    output_dir = "output"
    num_images = 2
    model_ids = [
        "runwayml/stable-diffusion-v1-5",
        "stabilityai/stable-diffusion-2-1",
    ]    
    model_dropdown = widgets.Dropdown(
        options=model_ids,
        value=model_ids[0],
        description="Select Model:",
    )    
    prompt_text = widgets.Text(
        value="",
        placeholder="Enter your prompt",
        description="Prompt:",
    )    
    num_images_slider = widgets.IntSlider(
        value=2,
        min=1,
        max=10,
        step=1,
        description="Num of Images:",
    )    
    image_url_text = widgets.Text(
        value="https://user-images.githubusercontent.com/786476/256401499-f010e3f8-6f8d-4e9f-9d1f-178d3571e7b9.png",
        placeholder="Enter an image URL",
        description="Image URL:",
    )
    enhance_checkbox = widgets.Checkbox(
        value=False,
        description="Auto enhance the prompt?",
        disabled=False,
        indent=False
    )
    layout = widgets.Layout(margin="0px 50px 10px 0px")
    button = widgets.Button(description="Generate Images!")
    left_box = VBox([model_dropdown, prompt_text, num_images_slider], layout=layout)
    right_box = VBox([image_url_text, enhance_checkbox], layout=layout)
    user_input_widgets = HBox([left_box, right_box], layout=layout)
    display(user_input_widgets)
    display(button)
    display(out)
    
    
    def on_submit(button):
        clear_output(wait=True)
        with out:
            print("Generating Images, once generated images will be saved `./output` dir, please wait...")
            selected_model_index = model_ids.index(model_dropdown.value)
            model_id = model_ids[selected_model_index]
            prompt = prompt_text.value
            num_images = num_images_slider.value
            image_url = image_url_text.value
            
            if not validators.url(image_url):
                print("The input is not a valid URL. Using the default URL instead.")
                image_url = "https://user-images.githubusercontent.com/786476/256401499-f010e3f8-6f8d-4e9f-9d1f-178d3571e7b9.png"       
            model = Img2ImgModel(model_id, device="xpu")
            enhancements = [
            "purple light",
            "dreaming",
            "cyberpunk",
            "ancient" ", rustic",
            "gothic",
            "historical",
            "punchy",
            "photo" "vivid colors",
            "4k",
            "bright",
            "exquisite",
            "painting",
            "art",
            "fantasy [,/organic]",
            "detailed",
            "trending in artstation fantasy",
            "electric",
            "night",
            ]
            
            if enhance_checkbox.value:
                prompt = prompt + " " + " ".join(random.sample(enhancements, 5))
                print(f"Using enhanced prompt: {prompt}")    
            try:
                start_time = time.time()
                os.makedirs(output_dir, exist_ok=True)
                model.generate_images(
                    prompt=prompt,
                    image_url=image_url,
                    num_images=num_images,
                )
                clear_output(wait=True)
                display_generated_images()
            except KeyboardInterrupt:
                print("\nUser interrupted image generation...")
            except Exception as e:
                print(f"An error occurred: {e}")
            finally:
                status = f"Complete generating {num_images} images in {time.time() - start_time:.2f} seconds."
                print(status)
    button.on_click(on_submit)

def display_generated_images(output_dir="output"):
    image_files = [f for f in os.listdir(output_dir) if f.endswith((".png", ".jpg"))]    
    num_images = len(image_files)
    grid_size = int(np.ceil(np.sqrt(num_images)))
    fig, axs = plt.subplots(grid_size, grid_size, figsize=(15, 15))    
    if num_images == 1:
        axs = np.array([[axs]])
    elif grid_size == 1:
        axs = np.array([axs])
    for ax, image_file in zip(axs.ravel(), image_files):
        img = mp_img.imread(os.path.join(output_dir, image_file))
        ax.imshow(img)
        ax.axis("off")  # Hide axes
    for ax in axs.ravel()[num_images:]:
        ax.axis("off") 
    plt.tight_layout()
    plt.show()

---

**Let"s Dive In! and Witness the Magi**

Ready to generate some amazing images? Just interact with the user interface below. Once you"ve set your preferences, click the "Generate Images!" button to witness the power of Stable Diffusion in action.

Once done, you"ll find the images generated based on your prompt. Presented in a grid format, they showcase the diverse interpretations and transformations the model inferred from your input.

Take a moment to marvel at the fusion of your creativity and the prowess of Stable Diffusion. Each image is a testament to the limitless possibilities this technology offers.


Go ahead, unleash your creativity!



In [10]:
image_to_image()

Button(description='Generate Images!', style=ButtonStyle())

Output()

## Reference and Guidelines for Models Used in This Notebook


### runwayml/stable-diffusion-v1-5
- **Model card:** [runwayml/stable-diffusion-v1-5](https://huggingface.co/runwayml/stable-diffusion-v1-5)
- **License:** CreativeML OpenRAIL M license
- **Reference:**
    ```bibtex
    @InProceedings{Rombach_2022_CVPR,
        author    = {Rombach, Robin and Blattmann, Andreas and Lorenz, Dominik and Esser, Patrick and Ommer, Bj\"orn},
        title     = {High-Resolution Image Synthesis With Latent Diffusion Models},
        booktitle = {Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition (CVPR)},
        month     = {June},
        year      = {2022},
        pages     = {10684-10695}
    }
    ```

### stabilityai/stable-diffusion-2-1
- **Model card:** [stabilityai/stable-diffusion-2-1](https://huggingface.co/stabilityai/stable-diffusion-2-1)
- **License:** CreativeML Open RAIL++-M License
- **Reference:**
    ```bibtex
    @InProceedings{Rombach_2022_CVPR,
        author    = {Rombach, Robin and Blattmann, Andreas and Lorenz, Dominik and Esser, Patrick and Ommer, Bj\"orn},
        title     = {High-Resolution Image Synthesis With Latent Diffusion Models},
        booktitle = {Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition (CVPR)},
        month     = {June},
        year      = {2022},
        pages     = {10684-10695}
    }
    ```

### Disclaimer for Using Stable Diffusion Models

The stable diffusion models provided here are powerful tools for high-resolution image synthesis, including text-to-image and image-to-image transformations. While they are designed to produce high-quality results, users should be aware of potential limitations:

- **Quality Variation:** The quality of generated images may vary based on the complexity of the input text or image, and the alignment with the model's training data.
- **Licensing and Usage Constraints:** Please carefully review the licensing information associated with each model to ensure compliance with all terms and conditions.
- **Ethical Considerations:** Consider the ethical implications of the generated content, especially in contexts that may involve sensitive or controversial subjects.

For detailed information on each model's capabilities, limitations, and best practices, please refer to the respective model cards and associated publications linked above.
